# Aplicación de Extracción de Datos Clave

## Introducción
* Crearemos una aplicación para **extraer información estructurada de texto no estructurado**. Imagina, por ejemplo, que quieres extraer el nombre, apellido y país de los usuarios que envían comentarios en la web de tu empresa.

## Conéctate con el archivo .env ubicado en el mismo directorio de este notebook

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Conéctate con un LLM

* NOTA: Dado que actualmente es el mejor LLM del mercado, usaremos OpenAI por defecto. Verás cómo conectarte con otros LLMs de código abierto como Llama3 o Mistral en una próxima lección.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Define qué información quieres extraer
* **Usaremos Pydantic para definir un esquema para extraer información personal**.
* Pydantic es una librería de Python utilizada para la validación de datos. Ayuda a asegurar que los datos que recibe tu programa coincidan con el formato que esperas, y proporciona mensajes de error útiles cuando los datos no cumplen con tus especificaciones. Esencialmente, Pydantic te permite hacer que las estructuras de datos en Python se ajusten a tipos y restricciones específicas, haciendo tu código más robusto y resistente a errores.
* **Documenta los atributos y el propio esquema**: Esta información se envía al LLM y se utiliza para mejorar la calidad de la extracción de información.
* ¡No fuerces al LLM a inventar información! **Importamos Optional para los atributos, permitiendo que el LLM devuelva None si no sabe la respuesta**.
* Cuando usas Optional en las anotaciones de tipo, indicas que una variable puede ser del tipo especificado o puede ser None.

#### Definamos los datos que queremos extraer de una persona.
* Observa abajo que es una buena práctica escribir un doc-string explicativo (comentarios) para ayudar al modelo de chat a entender qué datos queremos extraer.

In [3]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Información sobre una persona."""

    # ^ Doc-string para la entidad Person.
    # Este doc-string se envía al LLM como la descripción del esquema Person,
    # y puede ayudar a mejorar los resultados de la extracción.

    # Nota:
    # 1. Cada campo es `optional` -- esto permite que el modelo decline extraerlo.
    # 2. Cada campo tiene una `description` -- esta descripción es usada por el LLM.
    # Tener una buena descripción puede ayudar a mejorar los resultados de la extracción.
    name: Optional[str] = Field(
        default=None, description="El nombre de la persona"
    )
    lastname: Optional[str] = Field(
        default=None, description="El apellido de la persona si se conoce"
    )
    country: Optional[str] = Field(
        default=None, description="El país de la persona si se conoce"
    )

C:\Users\Juan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Define el extractor

Nuestro extractor será una cadena con la plantilla de prompt y un modelo de chat con las instrucciones de extracción.

In [4]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define un prompt personalizado para dar instrucciones y contexto adicional.
# 1) Puedes agregar ejemplos en la plantilla del prompt para mejorar la calidad de la extracción
# 2) Puedes introducir parámetros adicionales para tener en cuenta el contexto (por ejemplo, incluir metadatos
#    sobre el documento del que se extrajo el texto.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un algoritmo experto en extracción. "
            "Solo extrae información relevante del texto. "
            "Si no sabes el valor de un atributo solicitado para extraer, "
            "devuelve null para el valor del atributo.",
        ),
        ("human", "{text}"),
    ]
)

* Necesitamos usar un modelo que soporte llamadas a funciones/herramientas.
* Por favor revisa [la documentación](https://python.langchain.com/v0.2/docs/concepts/#function-tool-calling) para ver una lista de algunos modelos que pueden usarse con esta API.
* **Usaremos .with_structured_output() para añadir las instrucciones de extracción a nuestro modelo de chat**.

In [5]:
chain = prompt | llm.with_structured_output(schema=Person)

C:\Users\Juan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_openai\chat_models\base.py:1677: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
C:\Users\Juan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_openai\chat_models\base.py:1690: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


## Prueba la aplicación de extracción
Mira cómo la aplicación extrae el nombre, apellido y país de una reseña de usuario:

In [6]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"

In [7]:
chain.invoke({"text": comment})

Person(name='Sarah', lastname='Johnson', country='USA')

* **Ten en cuenta que esta capacidad de extracción es generativa**, lo que significa que nuestro modelo puede realizar una variedad de tareas más allá de lo esperado. Por ejemplo, el modelo podría inferir el género de un usuario a partir de su nombre, incluso cuando esta información no se proporciona explícitamente.

## Extracción de una lista de entidades en lugar de una sola entidad
* En proyectos reales probablemente trabajarás con un texto grande que incluya más de una reseña de usuario. **Podemos extraer los datos clave de varios usuarios anidando modelos de Pydantic**.
* Observa cómo la definición del modelo Data incluye el modelo Person. Esto se llama técnicamente “anidar” modelos.

In [8]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Información sobre una persona."""

    # ^ Doc-string para la entidad Person.
    # Este doc-string se envía al LLM como la descripción del esquema Person,
    # y puede ayudar a mejorar los resultados de la extracción.

    # Nota:
    # 1. Cada campo es `optional` -- esto permite que el modelo decline extraerlo.
    # 2. Cada campo tiene una `description` -- esta descripción es usada por el LLM.
    # Tener una buena descripción puede ayudar a mejorar los resultados de la extracción.
    name: Optional[str] = Field(
        default=None, description="El nombre de la persona"
    )
    lastname: Optional[str] = Field(
        default=None, description="El apellido de la persona si se conoce"
    )
    country: Optional[str] = Field(
        default=None, description="El país de la persona si se conoce"
    )

class Data(BaseModel):
    """Datos extraídos sobre personas."""

    # Crea un modelo para poder extraer múltiples entidades.
    people: List[Person]

Observa cómo ahora estamos usando el modelo Data con el llm:

In [9]:
chain = prompt | llm.with_structured_output(schema=Data)

C:\Users\Juan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_openai\chat_models\base.py:1677: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
C:\Users\Juan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_openai\chat_models\base.py:1690: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [10]:
comment = "I'm so impressed with this product! It has truly transformed how I approach my daily tasks. The quality exceeds my expectations, and the customer support is truly exceptional. I've already suggested it to all my colleagues and relatives. - Emily Clarke, Canada"

In [11]:
chain.invoke({"text": comment})

Data(people=[Person(name='Emily', lastname='Clarke', country='Canada')])

#### Veamos esto en acción con un texto que contiene varias reseñas.

In [12]:
# Ejemplo de texto de entrada que menciona a varias personas
text_input = """
Alice Johnson de Canadá revisó recientemente un libro que le encantó. Mientras tanto, Bob Smith de EE.UU. compartió sus ideas sobre el mismo libro en una reseña diferente. Ambas reseñas fueron muy perspicaces.
"""

# Invoca la cadena de procesamiento sobre el texto
response = chain.invoke({"text": text_input})

# Muestra los datos extraídos
response

Data(people=[Person(name='Alice', lastname='Johnson', country='Canadá'), Person(name='Bob', lastname='Smith', country='EE.UU.')])